Installation

In [ ]:
pip install scikit-learn

In [ ]:
pip install opencv-python

In [ ]:
pip install mediapipe

Required Libraries


In [ ]:

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize


from pandas.core.common import random_state
from sklearn.metrics import accuracy_score

import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt



Read csv files 

In [ ]:
file_1 = pd.read_csv('3d_distances.csv')
file_2 = pd.read_csv('angles.csv')
file_3 = pd.read_csv('labels.csv')
file_4 = pd.read_csv('landmarks.csv')
file_5 = pd.read_csv('xyz_distances.csv')

In [ ]:
merged_data = pd.merge(file_4,file_5,on='pose_id')
merged_data = pd.merge(merged_data,file_1,on='pose_id')
merged_data = pd.merge(merged_data,file_2,on='pose_id')

merged_data = pd.merge(merged_data,file_3,on='pose_id')

In [ ]:
merged_data.shape

In [ ]:
merged_data = merged_data.drop('pose_id',axis=1)
merged_data = merged_data.drop('right_knee_mid_hip_left_knee',axis=1)

In [ ]:
merged_data.shape

In [ ]:
merged_data['pose']

In [ ]:
merged_data.iloc[:0,:]

In [ ]:
merged_data.head()

In [ ]:
merged_data['pose'].unique()

In [ ]:
copy = merged_data

In [ ]:
merged_data['pose'].value_counts()

separate features,target_variable

In [ ]:
from sklearn.model_selection import train_test_split
x= merged_data.drop(['pose'], axis='columns')
y = merged_data['pose']

In [ ]:
encoder = LabelEncoder()
y = merged_data['pose']
y = encoder.fit_transform(y)

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_dict = {}
for idx, k in enumerate(class_weights):
    class_weights_dict[idx] = k

In [ ]:
class_weights_dict

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# x = x.drop(['pose_id'],axis=1)
x = scaler.fit_transform(x)
x

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(random_state=42)

classifier.fit(X_train,Y_train)

In [ ]:
y_pred = classifier.predict(X_test)
accuracy_rf = accuracy_score(Y_test,y_pred)

In [ ]:
print(accuracy_rf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

In [ ]:
from sklearn import svm
classifier_2 = svm.SVC()
classifier_2.fit(X_train,Y_train)

In [ ]:
y_pred = classifier_2.predict(X_test)
accuracy_svm = accuracy_score(Y_test,y_pred)
print(accuracy_svm)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier_3 = DecisionTreeClassifier(random_state = 0)
classifier_3.fit(X_train,Y_train)


In [ ]:
y_pred = classifier_3.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)


In [ ]:
print(classification_report(Y_test,y_pred))


In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)
mp_drawing = mp.solutions.drawing_utils

#Input : Image

In [ ]:
# import cv2
# sample_img  = cv2.imread(r"TestImages/pullups.jpeg") #uploaded this image to colab
# plt.figure(figsize = [10,10])                 #compress the size of image to 10x10
# plt.title("sample_Image");
# plt.axis('off');                    #removes the labels
# plt.imshow(sample_img[:,:,::-1]);  #converted bgr to rgb becoz to plt.show() expects rgb
# plt.show()                          #prints the image

#Input : Video 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

points = mp_pose.PoseLandmark       

data = []

for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data) 


def capture_image_and_landmarks():
    cap = cv2.VideoCapture(0)

    start_time = time.time()
    last_frame = None

    while time.time() - start_time <= 10:
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture frame.")
            break

        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(imgRGB)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark

            dict = {}
            for point, landmark in zip(points, landmarks):
                dict[point.name] = [landmark.x, landmark.y, landmark.z]

        last_frame = frame  

        cv2.imshow("Camera", frame)
        cv2.waitKey(1)

    cap.release()
    cv2.destroyAllWindows()

    return last_frame


last_frame = capture_image_and_landmarks()


if last_frame is not None:
    plt.title("Pose_Captured")
    plt.axis('off')                     
    plt.imshow(last_frame[:,:,::-1])     
    plt.show()                          
else:
    print("No frames captured.")

sample_img = last_frame


In [ ]:
results = pose.process(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))

In [ ]:
image_height, image_width, _ = sample_img.shape

if results.pose_landmarks:

    for i in range(11,20):

        print(f'{mp_pose.PoseLandmark(i).name}:')
        print(f'x: {results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].x * image_width}')
        print(f'y: {results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].y * image_height}')
        print(f'z: {results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].z * image_width}')
        print(f'visibility: {results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].visibility}\n')

In [ ]:
img_copy = sample_img.copy()
height, width, _ = img_copy.shape
landmarks = []

if results.pose_landmarks:
    mp_drawing.draw_landmarks(image=img_copy, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
    for landmark in results.pose_landmarks.landmark:

            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
   
    fig = plt.figure(figsize = [10, 10])

    # Display the output image with the landmarks drawn, also convert BGR to RGB for display.
    # plt.title("Output");
    # plt.axis('off');
    # plt.imshow(img_copy[:,:,::-1]);
    # plt.show()

In [ ]:
points = mp_pose.PoseLandmark      
     
data = []
for p in points:
  print(p)
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data) 

#When Running in google-colab -> uncomment code with %

In [ ]:
# from google.colab.patches import cv2_imshow   #-------GoogleColab Instruction line----------


In [ ]:
count = 0
temp = []
img = sample_img

        

imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
       

blackie = np.zeros(img.shape) 

results = pose.process(imgRGB)

if results.pose_landmarks:
        mp_drawing.draw_landmarks(blackie, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) # draw landmarks on blackie

        landmarks = results.pose_landmarks.landmark

        dict = {}
        for point, j in zip(points, landmarks):
                temp = temp + [j.x, j.y, j.z]
  
                key = point.name
                value = [temp[count], temp[count+1], temp[count+2]]
                dict[key] = value
                count = count + 1

# cv2.imshow('img',img)

# cv2.imshow('blackie',blackie)

# cv2_imshow(img)               #-------GoogleColab Instruction line----------
# cv2_imshow(blackie)           #-------GoogleColab Instruction line----------


Some additional distances that are not calculated by media_pose but are present in training dataset

In [ ]:
#xyz_distances
x_left_shoulder_left_wrist = dict['LEFT_WRIST'][0] - dict['LEFT_SHOULDER'][0] #(0.48930972814559937)-(-0.276510596275329)
y_left_shoulder_left_wrist = dict['LEFT_WRIST'][1] - dict['LEFT_SHOULDER'][1]#(0.143650084733963) -  (0.4948402941226959)
z_left_shoulder_left_wrist =  dict['LEFT_WRIST'][2] - dict['LEFT_SHOULDER'][2]#(-0.3052274286746979) - (0.14425188302993774)

x_right_shoulder_right_wrist = dict['RIGHT_WRIST'][0] - dict['RIGHT_SHOULDER'][0]#0.143650084733963 - 0.4948402941226959
y_right_shoulder_right_wrist = dict['RIGHT_WRIST'][1] - dict['RIGHT_SHOULDER'][1]#(-0.3052274286746979) -	0.14425188302993774
z_right_shoulder_right_wrist = dict['RIGHT_WRIST'][2] - dict['RIGHT_SHOULDER'][2]#0.48236632347106934 - (-0.3048369586467743)

x_left_hip_left_ankle	= dict['LEFT_ANKLE'][0] - dict['LEFT_HIP'][0]#0.5012699365615845 - (-0.12573319673538208)
y_left_hip_left_ankle	= dict['LEFT_ANKLE'][1] - dict['LEFT_HIP'][1]#0.1965891420841217 - 0.45757120847702026
z_left_hip_left_ankle	= dict['LEFT_ANKLE'][2] - dict['LEFT_HIP'][2]#(-0.24418438971042633)- 0.15223582088947296

x_right_hip_right_ankle	=  dict['RIGHT_ANKLE'][0] - dict['RIGHT_HIP'][0]
y_right_hip_right_ankle	= dict['RIGHT_ANKLE'][1] - dict['RIGHT_HIP'][1]
z_right_hip_right_ankle = dict['RIGHT_ANKLE'][2] - dict['RIGHT_HIP'][2]

x_left_hip_left_wrist	= dict['LEFT_WRIST'][0] - dict['LEFT_HIP'][0]
y_left_hip_left_wrist	=  dict['LEFT_WRIST'][1] - dict['LEFT_HIP'][1]
z_left_hip_left_wrist	=  dict['LEFT_WRIST'][2] - dict['LEFT_HIP'][2]

x_right_hip_right_wrist	= dict['RIGHT_WRIST'][0] - dict['RIGHT_HIP'][0]
y_right_hip_right_wrist	= dict['RIGHT_WRIST'][1] - dict['RIGHT_HIP'][1]
z_right_hip_right_wrist	= dict['RIGHT_WRIST'][2] - dict['RIGHT_HIP'][2]

x_left_shoulder_left_ankle	= dict['LEFT_ANKLE'][0] - dict['LEFT_SHOULDER'][0]
y_left_shoulder_left_ankle = dict['LEFT_ANKLE'][1] - dict['LEFT_SHOULDER'][1]
z_left_shoulder_left_ankle = dict['LEFT_ANKLE'][2] - dict['LEFT_SHOULDER'][2]

x_right_shoulder_right_ankle = dict['RIGHT_ANKLE'][0] - dict['RIGHT_SHOULDER'][0]
y_right_shoulder_right_ankle = dict['RIGHT_ANKLE'][1] - dict['RIGHT_SHOULDER'][1]
z_right_shoulder_right_ankle = dict['RIGHT_ANKLE'][2] - dict['RIGHT_SHOULDER'][2]

x_left_hip_right_wrist = dict['RIGHT_WRIST'][0] - dict['LEFT_HIP'][0]
y_left_hip_right_wrist = dict['RIGHT_WRIST'][1] - dict['LEFT_HIP'][1]
z_left_hip_right_wrist = dict['RIGHT_WRIST'][2] - dict['LEFT_HIP'][2]

x_right_hip_left_wrist = dict['LEFT_WRIST'][0] - dict['RIGHT_HIP'][0]
y_right_hip_left_wrist	= dict['LEFT_WRIST'][1] - dict['RIGHT_HIP'][1]
z_right_hip_left_wrist	= dict['LEFT_WRIST'][2] - dict['RIGHT_HIP'][2]

x_left_elbow_right_elbow = dict['RIGHT_ELBOW'][0] - dict['LEFT_ELBOW'][0]
y_left_elbow_right_elbow = dict['RIGHT_ELBOW'][1] - dict['LEFT_ELBOW'][1]
z_left_elbow_right_elbow = dict['RIGHT_ELBOW'][2] - dict['LEFT_ELBOW'][2]

x_left_knee_right_knee = dict['RIGHT_KNEE'][0] - dict['LEFT_KNEE'][0]
y_left_knee_right_knee	= dict['RIGHT_KNEE'][1] - dict['LEFT_KNEE'][1]
z_left_knee_right_knee	= dict['RIGHT_KNEE'][2] - dict['LEFT_KNEE'][2]

x_left_wrist_right_wrist = dict['RIGHT_WRIST'][0] - dict['LEFT_WRIST'][0]
y_left_wrist_right_wrisT = dict['RIGHT_WRIST'][1] - dict['LEFT_WRIST'][1]
z_left_wrist_right_wrist = dict['RIGHT_WRIST'][2] - dict['LEFT_WRIST'][2]

x_left_ankle_right_ankle = dict['RIGHT_ANKLE'][0] - dict['LEFT_ANKLE'][0]
y_left_ankle_right_ankle	= dict['RIGHT_ANKLE'][1] - dict['LEFT_ANKLE'][1]
z_left_ankle_right_ankle = dict['RIGHT_ANKLE'][2] - dict['LEFT_ANKLE'][2]

x_left_hip_avg_left_wrist_left_ankle = dict['LEFT_HIP'][0]-(dict['LEFT_WRIST'][0] + dict['LEFT_ANKLE'][0])/2
y_left_hip_avg_left_wrist_left_ankle = dict['LEFT_HIP'][1]-(dict['LEFT_WRIST'][1] + dict['LEFT_ANKLE'][1])/2
z_left_hip_avg_left_wrist_left_ankle = dict['LEFT_HIP'][2]-(dict['LEFT_WRIST'][2] + dict['LEFT_ANKLE'][2])/2

x_right_hip_avg_right_wrist_right_ankle = dict['RIGHT_HIP'][0]-(dict['RIGHT_WRIST'][0] + dict['RIGHT_ANKLE'][0])/2
y_right_hip_avg_right_wrist_right_ankle	= dict['RIGHT_HIP'][1]-(dict['RIGHT_WRIST'][1] + dict['RIGHT_ANKLE'][1])/2
z_right_hip_avg_right_wrist_right_ankle = dict['RIGHT_HIP'][2]-(dict['RIGHT_WRIST'][2] + dict['RIGHT_ANKLE'][2])/2


In [ ]:
#xyz_distances
c1 = dict['LEFT_WRIST'][0] - dict['LEFT_SHOULDER'][0]
c2 = dict['LEFT_WRIST'][1] - dict['LEFT_SHOULDER'][1]
c3 =  dict['LEFT_WRIST'][2] - dict['LEFT_SHOULDER'][2]

c4 = dict['RIGHT_WRIST'][0] - dict['RIGHT_SHOULDER'][0]
c5 = dict['RIGHT_WRIST'][1] - dict['RIGHT_SHOULDER'][1]
c6 = dict['RIGHT_WRIST'][2] - dict['RIGHT_SHOULDER'][2]

c7	= dict['LEFT_ANKLE'][0] - dict['LEFT_HIP'][0]
c8	= dict['LEFT_ANKLE'][1] - dict['LEFT_HIP'][1]
c9	= dict['LEFT_ANKLE'][2] - dict['LEFT_HIP'][2]

c10	=  dict['RIGHT_ANKLE'][0] - dict['RIGHT_HIP'][0]
c11	= dict['RIGHT_ANKLE'][1] - dict['RIGHT_HIP'][1]
c12 = dict['RIGHT_ANKLE'][2] - dict['RIGHT_HIP'][2]

c13	= dict['LEFT_WRIST'][0] - dict['LEFT_HIP'][0]
c14	=  dict['LEFT_WRIST'][1] - dict['LEFT_HIP'][1]
c15=  dict['LEFT_WRIST'][2] - dict['LEFT_HIP'][2]

c16	= dict['RIGHT_WRIST'][0] - dict['RIGHT_HIP'][0]
c17	= dict['RIGHT_WRIST'][1] - dict['RIGHT_HIP'][1]
c18= dict['RIGHT_WRIST'][2] - dict['RIGHT_HIP'][2]

c19	= dict['LEFT_ANKLE'][0] - dict['LEFT_SHOULDER'][0]
c20 = dict['LEFT_ANKLE'][1] - dict['LEFT_SHOULDER'][1]
c21 = dict['LEFT_ANKLE'][2] - dict['LEFT_SHOULDER'][2]

c22 = dict['RIGHT_ANKLE'][0] - dict['RIGHT_SHOULDER'][0]
c23 = dict['RIGHT_ANKLE'][1] - dict['RIGHT_SHOULDER'][1]
c24 = dict['RIGHT_ANKLE'][2] - dict['RIGHT_SHOULDER'][2]

c25 = dict['RIGHT_WRIST'][0] - dict['LEFT_HIP'][0]
c26 = dict['RIGHT_WRIST'][1] - dict['LEFT_HIP'][1]
c27 = dict['RIGHT_WRIST'][2] - dict['LEFT_HIP'][2]

c28 = dict['LEFT_WRIST'][0] - dict['RIGHT_HIP'][0]
c29	= dict['LEFT_WRIST'][1] - dict['RIGHT_HIP'][1]
c30	= dict['LEFT_WRIST'][2] - dict['RIGHT_HIP'][2]

c31 = dict['RIGHT_ELBOW'][0] - dict['LEFT_ELBOW'][0]
c32 = dict['RIGHT_ELBOW'][1] - dict['LEFT_ELBOW'][1]
c33 = dict['RIGHT_ELBOW'][2] - dict['LEFT_ELBOW'][2]

c34 = dict['RIGHT_KNEE'][0] - dict['LEFT_KNEE'][0]
c35	= dict['RIGHT_KNEE'][1] - dict['LEFT_KNEE'][1]
c36	= dict['RIGHT_KNEE'][2] - dict['LEFT_KNEE'][2]

c37 = dict['RIGHT_WRIST'][0] - dict['LEFT_WRIST'][0]
c38 = dict['RIGHT_WRIST'][1] - dict['LEFT_WRIST'][1]
c39 = dict['RIGHT_WRIST'][2] - dict['LEFT_WRIST'][2]

c40 = dict['RIGHT_ANKLE'][0] - dict['LEFT_ANKLE'][0]
c41	= dict['RIGHT_ANKLE'][1] - dict['LEFT_ANKLE'][1]
c42= dict['RIGHT_ANKLE'][2] - dict['LEFT_ANKLE'][2]

c43 = dict['LEFT_HIP'][0]-(dict['LEFT_WRIST'][0] + dict['LEFT_ANKLE'][0])/2
c44 = dict['LEFT_HIP'][1]-(dict['LEFT_WRIST'][1] + dict['LEFT_ANKLE'][1])/2
c45 = dict['LEFT_HIP'][2]-(dict['LEFT_WRIST'][2] + dict['LEFT_ANKLE'][2])/2

c46 = dict['RIGHT_HIP'][0]-(dict['RIGHT_WRIST'][0] + dict['RIGHT_ANKLE'][0])/2
c47	= dict['RIGHT_HIP'][1]-(dict['RIGHT_WRIST'][1] + dict['RIGHT_ANKLE'][1])/2
c48 = dict['RIGHT_HIP'][2]-(dict['RIGHT_WRIST'][2] + dict['RIGHT_ANKLE'][2])/2


In [ ]:
#3d_distances

c49 = np.sqrt((dict['LEFT_WRIST'][0] - dict['LEFT_SHOULDER'][0])**2 + (dict['LEFT_WRIST'][1] - dict['LEFT_SHOULDER'][1])**2 + (dict['LEFT_WRIST'][2] - dict['LEFT_SHOULDER'][2])**2)
c50 = np.sqrt((dict['RIGHT_WRIST'][0] - dict['RIGHT_SHOULDER'][0])**2 + (dict['RIGHT_WRIST'][1] - dict['RIGHT_SHOULDER'][1])**2 + (dict['RIGHT_WRIST'][2] - dict['RIGHT_SHOULDER'][2])**2)
c51= np.sqrt((dict['LEFT_ANKLE'][0] - dict['LEFT_HIP'][0])**2 + (dict['LEFT_ANKLE'][1] - dict['LEFT_HIP'][1])**2 + (dict['LEFT_ANKLE'][2] - dict['LEFT_HIP'][2])**2)
c52= np.sqrt((dict['RIGHT_ANKLE'][0] - dict['RIGHT_HIP'][0])**2 + (dict['RIGHT_ANKLE'][1] - dict['RIGHT_HIP'][1])**2 + (dict['RIGHT_ANKLE'][2] - dict['RIGHT_HIP'][2])**2)
c53	= np.sqrt((dict['LEFT_WRIST'][0] - dict['LEFT_HIP'][0])**2 + (dict['LEFT_WRIST'][1] - dict['LEFT_HIP'][1])**2 + (dict['LEFT_WRIST'][2] - dict['LEFT_HIP'][2])**2)
c54	= np.sqrt((dict['RIGHT_WRIST'][0] - dict['RIGHT_HIP'][0])**2 + (dict['RIGHT_WRIST'][1] - dict['RIGHT_HIP'][1])**2 + (dict['RIGHT_WRIST'][2] - dict['RIGHT_HIP'][2])**2)
c55 = np.sqrt((dict['LEFT_ANKLE'][0] - dict['LEFT_SHOULDER'][0])**2 + (dict['LEFT_ANKLE'][1] - dict['LEFT_SHOULDER'][1])**2 + (dict['LEFT_ANKLE'][2] - dict['LEFT_SHOULDER'][2])**2)
c56 = np.sqrt((dict['RIGHT_ANKLE'][0] - dict['RIGHT_SHOULDER'][0])**2 + (dict['RIGHT_ANKLE'][1] - dict['RIGHT_SHOULDER'][1])**2 + (dict['RIGHT_ANKLE'][2] - dict['RIGHT_SHOULDER'][2])**2)
c57 = np.sqrt((dict['RIGHT_WRIST'][0] - dict['LEFT_HIP'][0])**2 + (dict['RIGHT_WRIST'][1] - dict['LEFT_HIP'][1])**2 + (dict['RIGHT_WRIST'][2] - dict['LEFT_HIP'][2])**2)
c58 = np.sqrt((dict['LEFT_WRIST'][0] - dict['RIGHT_HIP'][0])**2 + (dict['LEFT_WRIST'][1] - dict['RIGHT_HIP'][1])**2 + (dict['LEFT_WRIST'][2] - dict['RIGHT_HIP'][2])**2)
c59= np.sqrt((dict['RIGHT_ELBOW'][0] - dict['LEFT_ELBOW'][0])**2 + (dict['RIGHT_ELBOW'][1] - dict['LEFT_ELBOW'][1])**2 + (dict['RIGHT_ELBOW'][2] - dict['LEFT_ELBOW'][2])**2)
c60 = np.sqrt((dict['RIGHT_KNEE'][0] - dict['LEFT_KNEE'][0])**2 + (dict['RIGHT_KNEE'][1] - dict['LEFT_KNEE'][1])**2 + (dict['RIGHT_KNEE'][2] - dict['LEFT_KNEE'][2])**2)
c61 = np.sqrt((dict['RIGHT_WRIST'][0] - dict['LEFT_WRIST'][0])**2 + (dict['RIGHT_WRIST'][1] - dict['LEFT_WRIST'][1])**2 + (dict['RIGHT_WRIST'][2] - dict['LEFT_WRIST'][2])**2)
c62 = np.sqrt((dict['RIGHT_ANKLE'][0] - dict['LEFT_ANKLE'][0])**2 + (dict['RIGHT_ANKLE'][1] - dict['LEFT_ANKLE'][1])**2 + (dict['RIGHT_ANKLE'][2] - dict['LEFT_ANKLE'][2])**2)

x_avg = (dict['LEFT_WRIST'][0] + dict['LEFT_ANKLE'][0])/2
y_avg = (dict['LEFT_WRIST'][1] + dict['LEFT_ANKLE'][1])/2
z_avg = (dict['LEFT_WRIST'][2] + dict['LEFT_ANKLE'][2])/2

c63 = np.sqrt((x_avg - dict['LEFT_HIP'][0])**2 + (y_avg - dict['LEFT_HIP'][1])**2 + (z_avg - dict['LEFT_HIP'][2])**2)



x_avg = (dict['RIGHT_WRIST'][0] + dict['LEFT_ANKLE'][0])/2
y_avg = (dict['RIGHT_WRIST'][1] + dict['LEFT_ANKLE'][1])/2
z_avg = (dict['RIGHT_WRIST'][2] + dict['LEFT_ANKLE'][2])/2
c64 = np.sqrt((x_avg - dict['RIGHT_HIP'][0])**2 + (y_avg - dict['RIGHT_HIP'][1])**2 + (z_avg - dict['RIGHT_HIP'][2])**2)

In [ ]:
#angles
import math
import numpy as np

Ax = dict['RIGHT_SHOULDER'][0] - dict['RIGHT_ELBOW'][0]
Ay = dict['RIGHT_SHOULDER'][1] - dict['RIGHT_ELBOW'][1]
Az = dict['RIGHT_SHOULDER'][2] - dict['RIGHT_ELBOW'][2]
Bx = dict['RIGHT_SHOULDER'][0] - dict['RIGHT_HIP'][0]
By = dict['RIGHT_SHOULDER'][1] - dict['RIGHT_HIP'][1]
Bz = dict['RIGHT_SHOULDER'][2] - dict['RIGHT_HIP'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c65= np.degrees(angle)

Ax = dict['LEFT_SHOULDER'][0] - dict['LEFT_ELBOW'][0]
Ay = dict['LEFT_SHOULDER'][1] - dict['LEFT_ELBOW'][1]
Az = dict['LEFT_SHOULDER'][2] - dict['LEFT_ELBOW'][2]
Bx = dict['LEFT_SHOULDER'][0] - dict['LEFT_HIP'][0]
By = dict['LEFT_SHOULDER'][1] - dict['LEFT_HIP'][1]
Bz = dict['LEFT_SHOULDER'][2] - dict['LEFT_HIP'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c66	= np.degrees(angle)

Ax = (dict['LEFT_HIP'][0] + dict['RIGHT_HIP'][0])/2 - dict['RIGHT_KNEE'][0]
Ay =(dict['LEFT_HIP'][1] + dict['RIGHT_HIP'][1])/2 - dict['RIGHT_KNEE'][1]
Az = (dict['LEFT_HIP'][2] + dict['RIGHT_HIP'][2])/2 - dict['RIGHT_KNEE'][2]
Bx = (dict['LEFT_HIP'][0] + dict['RIGHT_HIP'][0])/2 - dict['LEFT_KNEE'][0]
By = (dict['LEFT_HIP'][1] + dict['RIGHT_HIP'][1])/2 - dict['LEFT_KNEE'][1]
Bz = (dict['LEFT_HIP'][2] + dict['RIGHT_HIP'][2])/2 - dict['LEFT_KNEE'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c67 = np.degrees(angle)



Ax = dict['RIGHT_KNEE'][0] - dict['RIGHT_HIP'][0]
Ay = dict['RIGHT_KNEE'][1] - dict['RIGHT_HIP'][1]
Az = dict['RIGHT_KNEE'][2] - dict['RIGHT_HIP'][2]
Bx = dict['RIGHT_KNEE'][0] - dict['RIGHT_ANKLE'][0]
By = dict['RIGHT_KNEE'][1] - dict['RIGHT_ANKLE'][1]
Bz = dict['RIGHT_KNEE'][2] - dict['RIGHT_ANKLE'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c68 = np.degrees(angle)

Ax = dict['LEFT_KNEE'][0] - dict['LEFT_HIP'][0]
Ay = dict['LEFT_KNEE'][1] - dict['LEFT_HIP'][1]
Az = dict['LEFT_KNEE'][2] - dict['LEFT_HIP'][2]
Bx = dict['LEFT_KNEE'][0] - dict['LEFT_ANKLE'][0]
By = dict['LEFT_KNEE'][1] - dict['LEFT_ANKLE'][1]
Bz = dict['LEFT_KNEE'][2] - dict['LEFT_ANKLE'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c69=  np.degrees(angle)


Ax = dict['RIGHT_ELBOW'][0] - dict['RIGHT_WRIST'][0]
Ay = dict['RIGHT_ELBOW'][1] - dict['RIGHT_WRIST'][1]
Az = dict['RIGHT_ELBOW'][2] - dict['RIGHT_WRIST'][2]
Bx = dict['RIGHT_ELBOW'][0] - dict['RIGHT_SHOULDER'][0]
By = dict['RIGHT_ELBOW'][1] - dict['RIGHT_SHOULDER'][1]
Bz = dict['RIGHT_ELBOW'][2] - dict['RIGHT_SHOULDER'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c70 = np.degrees(angle)

Ax = dict['LEFT_ELBOW'][0] - dict['LEFT_WRIST'][0]
Ay = dict['LEFT_ELBOW'][1] - dict['LEFT_WRIST'][1]
Az = dict['LEFT_ELBOW'][2] - dict['LEFT_WRIST'][2]
Bx = dict['LEFT_ELBOW'][0] - dict['LEFT_SHOULDER'][0]
By = dict['LEFT_ELBOW'][1] - dict['LEFT_SHOULDER'][1]
Bz = dict['LEFT_ELBOW'][2] - dict['LEFT_SHOULDER'][2]
AdotB = (Ax * Bx) + (Ay * By) + (Az * Bz)
modA = np.sqrt(Ax**2 + Ay**2 + Az**2)
modB = np.sqrt(Bx**2 + By**2 + Bz**2)
cos_angle = AdotB / (modA * modB)
angle = np.arccos(cos_angle)
c71 = np.degrees(angle)


In [ ]:
for i in range(1,72):
  if(i!=67):
    temp.append(locals()['c' + str(i)])

convert list to numpy array and reshape it to 2D array

In [ ]:

temp = np.array(temp)         
temp = temp.reshape(-1, 169)  
print(temp)

In [ ]:
y_pred = classifier.predict(temp)

Model Predicted Output

In [ ]:
y_pred_class_names = encoder.inverse_transform(y_pred)
y_pred_class_names